# Database Query and Heatmap (V2)

Library Initialization

In [1]:
from Modules.database_gen_sqlite import gen_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Database Generation Parameters

In [2]:
wall_width, wall_height = 120, 360
num_studs = 50
base_name = "test1"

Generate Database

In [3]:
gen_data(base_name, wall_width, wall_height, num_studs, True)

0

Extract Data from database